## This notebook presents different ways to present and interpret stock data
<br/> <br/>
#### To make the most of this notebook copy the github link above and follow the prompts to notebook viewer here: https://www.tutorialspoint.com/jupyter/sharing_jupyter_notebook_using_github_and_nbviewer.htm
#### Make sure you click the 'binder' button on nbviewer to interact with the plots



In [ ]:
# Import generic system packages for data manipulation 
import os
import pandas as pd
import numpy as np
import datetime as dt
import numpy as np

# Import packages for plots
import matplotlib.pyplot as plt 
import panel as pn
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

# Import packages for data visualisation
import mplfinance as mpf
from dotenv import load_dotenv
import cufflinks as cf
from IPython.display import display
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import holoviews as hv
import seaborn as sns

# Activate set up
load_dotenv()     
%matplotlib inline 
cf.set_config_file(theme='solar',sharing='public',offline=True) #List of Cufflinks Themes :  ['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']


### Get stock data

In [ ]:
#Read in alpaca keys and set up API call
alpaca_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_API_SECRET_KEY')
api = tradeapi.REST(alpaca_key, 
                    alpaca_secret_key,
                    api_version='V2')

#Define variables
tickers = ['GOOG', 'FB', 'AMZN', 'AAPL', 'TSLA']

tf = '1D'

start_date = pd.Timestamp('2018-1-1', tz='America/New_York').isoformat()

end_date = pd.Timestamp('2021-1-1', tz='America/New_York').isoformat()

wt = [0.25, 0.25, 0.25, 0.25]

num_sim = 100

num_trade_days=252*2

#Define parameters to get stock data from Alpaca api
tickers_df = api.get_barset(tickers, timeframe=tf, start=start_date, end=end_date, limit=1000).df

#Define paramters to run Monte-Carlo Simulation
close_prices = tickers_df.loc(axis=1)[:,'close']

daily_returns = close_prices.pct_change().dropna()
#print(f"daily ret shape: {daily_returns.shape}")

correlation = daily_returns.corr()

portfolio_cum_return = (1+daily_returns).cumprod()  
#print(f"Cum ret shape: {portfolio_cum_return.shape}")

roll_std = daily_returns.rolling(window=5).std()
#print(f"Roll std shape: {roll_std.shape}")

sharpe_ratio = (daily_returns.mean()*252) / (daily_returns.std()*np.sqrt(252))
#print(f"Sharpe ratio shape: {sharpe_ratio.shape}")


### Present OHLC data using candlesticks

In [ ]:
# Let's use Googole's and Facebook's OHLC 
ohlc1 = tickers_df["GOOG"]

ohlc_plot1 = ohlc1.iplot(kind='candle',
                         title='2018 - 2021 stock OHLC movement - Google',
                         yTitle='Price',
                         keys=True)


ohlc2 = tickers_df["FB"]
ohlc_plot2 = ohlc2.iplot(kind='candle',
                         title='2018 - 2021 stock OHLC movement - Facebook',
                         yTitle='Price',
                         keys=True)

In [ ]:
mpf_plot = mpf.plot(ohlc1,
                    type='candle',
                    mav=(10, 15, 20),
                    volume=True,
                    style='mike',
                    title="Google OHLC",
                    warn_too_much_data=5000)

mpf_plot = mpf.plot(ohlc2,
                    type='candle',
                    mav=(10, 15, 20),
                    volume=True,
                    style='mike',
                    title= "Facebook OHLC",
                    warn_too_much_data=5000)

### Plot rolling standard deviations as line plots

In [ ]:
# Format dataframe
roll_std.columns = roll_std.columns.droplevel(1)
roll_std.dropna(inplace=True)
roll_std.head()

In [ ]:
#Use hv.plot()
roll_std.hvplot(title="5-day rolling standard deviation",
                width=1200,
                height=500)

In [ ]:
#Use cufflinks' iplot
roll_std.iplot(kind='line',
               fill=True,
               yTitle='Standard deviation')

### Plot rolling standard deviations as line plots

In [ ]:
portfolio_cum_return.columns = portfolio_cum_return.columns.droplevel(1)
portfolio_cum_return.dropna(inplace=True)
portfolio_cum_return.head()

In [ ]:
# Using hvplot
AAPL_hvplot = portfolio_cum_return[["AAPL"]].hvplot(x='Time (years)',
                                                    color='red',
                                                    label="AAPL")



AMZN_hvplot = portfolio_cum_return[["AMZN"]].hvplot(x='Time (years)',
                                                    color='blue',
                                                    label="AMZN")


FB_hvplot = portfolio_cum_return[["FB"]].hvplot(x='Time (years)',
                                                color='black',
                                                label="FB")


GOOG_hvplot = portfolio_cum_return[["GOOG"]].hvplot(x='Time (years)',
                                                    color='green',
                                                    label="GOOG")
                                                  

TSLA_hvplot = portfolio_cum_return[["TSLA"]].hvplot(x='Time (years)',
                                                    color='yellow',
                                                    label="TSLA")

comb = (AAPL_hvplot * AMZN_hvplot * FB_hvplot * GOOG_hvplot * TSLA_hvplot).opts(legend_position='right',
                                                                                width=1200,
                                                                                height=400)
comb

In [ ]:
daily_returns.columns = daily_returns.columns.droplevel(1)
daily_returns.dropna(inplace=True)
daily_returns.head()

In [ ]:
# Plot cumulative returns
# Calculate weighted returns (equal weighting)
aapl_returns = daily_returns['AAPL'] * 0.2
amzn_returns = daily_returns['AMZN'] * 0.2
fb_returns = daily_returns['FB'] * 0.2
goog_returns = daily_returns['GOOG'] * 0.2
tsla_returns = daily_returns['TSLA'] * 0.2

# Calculate cumulative returns of all portfolios
cumlative_returns_aapl = (1+aapl_returns).cumprod()
cumlative_returns_amzn = (1+amzn_returns).cumprod()
cumlative_returns_fb = (1+fb_returns).cumprod()
cumlative_returns_goog = (1+goog_returns).cumprod()
cumlative_returns_tsla = (1+tsla_returns).cumprod()

# Plot cumulative returns
plot1 = cumlative_returns_aapl.plot(figsize=(10, 7), color='y', linestyle='dashed', title='Comparing all stocks')
cumlative_returns_amzn.plot(ax=plot1, color='r', linestyle='dotted', alpha=0.9)
cumlative_returns_fb.plot(ax=plot1, color='g', linestyle='dashdot', alpha=0.8)
cumlative_returns_goog.plot(ax=plot1, color='b', linestyle='dotted', alpha=0.7)
cumlative_returns_tsla.plot(ax=plot1, color='m', linestyle='dotted', alpha=0.6)

plot1.legend(['AAPL', 'AMZN', 'FB', 'GOOG', 'TSLA'])
plot1.set_xlabel('Year')
plot1.set_ylabel('Cumulative returns')

In [ ]:
#Using cuff links
portfolio_cum_return.iplot(kind='line',
                           fill=True,
                           yTitle="Cumulative returns")

### Plotting correlations

In [ ]:
correlation.index = correlation.index.droplevel(1)
correlation.columns = correlation.columns.droplevel(1)
correlation

In [ ]:
#Using iplot() (https://stackoverflow.com/questions/42310243/cufflinks-how-to-set-upper-and-lower-value-for-the-colorscale)
correlation.iplot(kind='heatmap',
                  mode='markers',
                  title="Correlation plot",
                  colorscale='spectral')

In [ ]:
# Usign hvplot
heatmap_plot = correlation.hvplot.heatmap(ymarks_hover_color='cyan',
                                          hover_color='cyan',
                                          alpha=0.9,
                                          colorbar=True,
                                          cmap='bjy')

heatmap_plot

In [ ]:
# Using Seaborn
matrix = np.triu(correlation)
cmap = sns.diverging_palette(230, 20, as_cmap=True)
h = plt.axes()

plot = sns.heatmap(correlation, 
                   vmin=-1, 
                   vmax=1, 
                   annot=True,
                   cmap=cmap, 
                   mask=matrix)


sns.set(rc={'figure.figsize':(10,7)})
h.set_title('Correlation')